In [144]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

import time

In [145]:
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")  #for windows only
#chrome_options.add_argument("--no-sandbox") # linux only
#chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works
#利用一个已经打开的浏览器进行继续控制,chrome版本需要对应
#  chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\selenum\AutomationProfile"
chrome_options.add_experimental_option('debuggerAddress',"127.0.0.1:9222")

#driver = webdriver.Chrome('/path/to/chromedriver')  #chromedriver.exe在程序同一目录下，需要和chrome版本一致
driver = webdriver.Chrome(options=chrome_options)


In [146]:
# booking_url = 'https://www.maersk.com.cn/ibooking/'
# driver.get(booking_url)
# time.sleep(8)

In [147]:
# user_name = 'Olhy_08'
# user_pass = 'Olhy1818'

# current_month = '8'
# expected_month = '8'
# expected_date = '21'

# commodity = 'Toys, games'
# container = {
#                     'size' :'40 Dry High',
#                     'quantity': 1,
#                     'weight': 18000

#                     }

# start_port = 'Ningbo (Zhejiang), China'
# end_port = 'Tema, Ghana'

# start_xpath = '//*[@id="container"]/div/section/form/div[5]/div[1]/div/div/fieldset/div[2]/span/div/div/a'
# end_xpath = '//*[@id="container"]/div/section/form/div[5]/div[2]/div/div/fieldset/div[2]/span/div/div/a'



In [148]:
#封装成函数
#start_xpath = '//*[@id="container"]/div/section/form/div[5]/div[1]/div/div/fieldset/div[2]/span/div/div/a'
#end_xpath = '//*[@id="container"]/div/section/form/div[5]/div[2]/div/div/fieldset/div[2]/span/div/div/a'

def select_port(xpath,port):
    select = driver.find_elements_by_xpath(xpath)
    for i,x in enumerate(select):
        if x.text == port:
            # print(x.text)
            # print("找到结果：" ,i)
            action = ActionChains(driver)
            action.move_to_element(x).click().perform()
            break

#函数移动到点击
def move_and_click(xpath, next_tag, expected_data):
    table_trs = driver.find_elements_by_xpath(xpath)
    for index,tr in enumerate(table_trs):
        tds = tr.find_elements_by_tag_name(next_tag)
        for key, td in enumerate(tds):
            if td.text == expected_data:
                print(td.text)
                action = ActionChains(driver)
                action.move_to_element(td).click().perform() 
                break

#updated by sam: 2021.08.03
# 处理日期选择

def datepicker(expected_month,expected_date):
    if expected_month == str(time.localtime()[1]):
        driver.find_element_by_id('pseudo-date-input-shipmentDate').click()
    else:
        driver.find_element_by_id('pseudo-date-input-shipmentDate').click()  
        driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div/a[2]').click()
    
    time.sleep(0.5)

    tr_xpath = '//*[@id="ui-datepicker-div"]/table/tbody/tr'
    move_and_click(tr_xpath, 'td', expected_date)

    # table_trs = driver.find_elements_by_xpath(tr_xpath)
    # for index,tr in enumerate(table_trs):
    #     tds = tr.find_elements_by_tag_name('td')
    #     for key, td in enumerate(tds):
    #         if td.text == expected_date:
    #             print(td.text)
    #             action = ActionChains(driver)
    #             action.move_to_element(td).click().perform() 
    #             break

#封装成函数
def container_size(container):
    xpath = '//*[@id="container-item-0"]/div[1]/div[1]/div/span/div/div'

    driver.find_element_by_id('cargoSectionDiv').click()
    # time.sleep(1)
    # driver.find_element_by_id('container-item-0').click()
    # driver.find_element_by_xpath('//*[@id="containerSectionDiv"]/div/div/')
    driver.find_element_by_id("containerSectionDiv").find_elements_by_tag_name('input')[1].click()
    time.sleep(0.5)
    move_and_click(xpath,'a', container['size'])

    # divs = driver.find_elements_by_xpath(xpath)
    # for index,div in enumerate(divs):
    #     a_tags = div.find_elements_by_tag_name('a')
    #     print(len(a_tags))
    #     for key, a_tag in enumerate(a_tags):
    #         print(a_tag.text)
    #         if a_tag.text == container['size']:
    #             print('找到了')
    #             action = ActionChains(driver)
    #             action.move_to_element(a_tag).click().perform() 
    #             break

In [149]:
def start_to_end(start_port,end_port):
    start_xpath = '//*[@id="container"]/div/section/form/div[5]/div[1]/div/div/fieldset/div[2]/span/div/div/a'
    end_xpath = '//*[@id="container"]/div/section/form/div[5]/div[2]/div/div/fieldset/div[2]/span/div/div/a'
    # 选择始发地
    driver.find_element_by_id('origin').send_keys(start_port + Keys.RETURN)  
    time.sleep(3)

    select_port(start_xpath,start_port)
    time.sleep(0.5)


    # #目的地
    driver.find_element_by_id('destination').send_keys(end_port + Keys.RETURN)
    time.sleep(3)
    select_port(end_xpath,end_port)
    time.sleep(1)

 # #合约人
def contract_owner():
    driver.find_element_by_xpath('//*[@id="priceOwner"]/div/div[2]/div[3]/button[2]').click()  
    driver.implicitly_wait(2)
    driver.find_element_by_css_selector('body > div.overlay.overlay--transparent > div > div > div > div:nth-child(2) > div > div.option-item__button > button').click()

#日期选择
def date_pick(expected_month,expected_date):
    #离港时间选择
    datepicker(expected_month,expected_date)


def carbinet_info(commodity,container):
    #鼠标移动到底部
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)

    #品名类别
    xpath = '//*[@id="cargo-item-0"]/div/div[1]/div[1]/div[1]/span/div/div/a'
    driver.find_element_by_id('commodity').send_keys(commodity + Keys.RETURN)
    time.sleep(0.5)
    select_port(xpath,commodity)

    # driver.find_element_by_id('containerCount-00').send_keys(container['quantity'])
    # time.sleep(0.5)

    #集装箱重量
    driver.find_element_by_id('containerWeight-00').send_keys(container['weight'])
    time.sleep(0.5)

    #箱型尺寸
    time.sleep(1.5)
    container_size(container)
    


def continue_result():
    #continue继续
    driver.find_element_by_id('continue').click()
    time.sleep(6)

    #查询结果确认
    result = driver.find_element_by_css_selector('body > div.overlay.overlay--transparent.sc-overlay > div > div > div > form > h2')
    if '没有找到舱保' in result.text:
        print('没有找到舱保')
        return False
    else:
        print('CONGRATULATION!')
        return True

def close():
    button_close_selector = 'body > div.overlay.overlay--transparent.sc-overlay > div > button'
    driver.find_element_by_css_selector(button_close_selector).click()
    time.sleep(2)


In [150]:
# if '__name__' == '__main__':
booking_url = 'https://www.maersk.com.cn/ibooking/'
driver.get(booking_url)
time.sleep(8)

expected_date_lst = ['16','18','20']

user_name = 'Olhy_08'
user_pass = 'Olhy1818'

# current_month = '8'
expected_month = '8'
# expected_date = expected_date

commodity = 'Toys, games'
container = {
                    'size' :'40 Dry High',
                    'quantity': 1,
                    'weight': 18000

                    }

start_port = 'Ningbo (Zhejiang), China'
end_port = 'Tema, Ghana'

#run selections:
start_to_end(start_port,end_port)
contract_owner()

carbinet_info(commodity,container)

#启用日期轮询
for expected_date in expected_date_lst:
    date_pick(expected_month,expected_date)
    
    print(expected_date)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    bool = continue_result()
    if not bool:
        close()
        driver.execute_script("window.scrollTo('#pseudo-date-input-shipmentDate',200)")
        continue
    else:
        print('恭喜找到舱保,请付款！')
else:
    print("无法找到舱保")
    


    


40 Dry High
16
16
没有找到舱保
18
18
没有找到舱保
20
20
没有找到舱保
无法找到舱保


In [151]:
import time
month = time.localtime()[1]
type(month)

int

In [152]:
#箱型尺寸选择
# #尺寸封装成函数
# #品名类别
# xpath = '//*[@id="cargo-item-0"]/div/div[1]/div[1]/div[1]/span/div/div/a'
# driver.find_element_by_id('commodity').send_keys(commodity + Keys.RETURN)
# time.sleep(0.5)
# select_port(xpath,commodity)

# # driver.find_element_by_id('cargoSectionDiv').click()
# # driver.find_element_by_id('container-item-0').click()
# # action = ActionChains(driver)
# # time.sleep(0.5)
# container_size()
# action.move_to_element(select).click().perform()

# xpath = '//*[@id="cargo-item-0"]/div/div[1]/div[1]/div[1]/span/div/div/a'
# driver.find_element_by_id('commodity').send_keys(commodity + Keys.RETURN)
# time.sleep(0.5)
# select_port(xpath,commodity)

